In [6]:
import pandas as pd

# Import database from excel sheet. It is organized in columns [variable | unit | symbol | value]
# You can set index col (you can take symbol for cleaner formulas)
df = pd.read_excel("../../database/ml_projekty.xlsx", sheet_name="Base", index_col="index", skiprows=1)

display(df.head(10))
# to get value us df.at[<index>, "value"]
# df.at["w_e","value"]
# df.at[]


SyntaxError: invalid syntax (266019986.py, line 10)